In [46]:
import os
import mne
import pickle as pkl
import numpy as np
import pandas as pd
import spkit as sp
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from glob import glob
from scipy.signal import welch
from mne_faster import (
    find_bad_channels,
    find_bad_epochs,
    find_bad_components,
    find_bad_channels_in_epochs,
)
from sklearn.feature_selection import mutual_info_classif

In [47]:
subjectid = '28'

### Getting paths of files

In [48]:
#Run for all subs data
all_files_path = glob("../final-dataset/*.set")

In [49]:
display(len(all_files_path))

348

In [50]:
if len(all_files_path) == 348:
  taskid = 10
elif len(all_files_path) == 120:
  taskid = 8

In [51]:
sub_file_paths = [i for i in all_files_path if os.path.basename(i)[4:6] == subjectid]

In [52]:
rest_files_path = [i for i in sub_file_paths if os.path.basename(i)[taskid] == "r"]

In [53]:
zerob_files_path = [i for i in sub_file_paths if os.path.basename(i)[taskid] == "0"]

In [54]:
oneb_files_path = [i for i in sub_file_paths if os.path.basename(i)[taskid] == "1"]

In [55]:
twob_files_path = [i for i in sub_file_paths if os.path.basename(i)[taskid] == "2"]

In [56]:
display(
  len(rest_files_path),
  # len(zerob_files_path),
  # len(oneb_files_path),
  len(twob_files_path)
)

3

3

### Data Preprocessing

In [57]:
def read_filter(filepath):
    # reading data from set file
    raw = mne.io.read_raw_eeglab(filepath, preload=True, verbose=False)

    # dropping ecg1 and cz channels as cz is not present in all samples
    raw.drop_channels(
        ch_names=["ECG1", "Cz"],
        on_missing="ignore",  # can be set to raise, warn or ignore
    )

    # applying high pass filter with threshold 1hz
    raw = raw.filter(
        l_freq=1,
        h_freq=None,
        n_jobs=4,
        verbose=False,
    )

    # notch filtering with threshold 50hz
    raw = raw.notch_filter(
        freqs=50,
        verbose=False,
    )

    return raw

In [58]:
def get_epochs(raw, task):
  if task == "r":
    epochs = mne.make_fixed_length_epochs(
      raw=raw,
      duration=1,
      preload=True,
      overlap=0.59,
      verbose=False,
    )

  else:
      include_annots = [f"6{task}21", f"6{task}22"]

      # include only the specified events in the annotations
      include_mask = [
          annot in include_annots for annot in raw.annotations.description
      ]

      included_annotations = mne.Annotations(
          onset=raw.annotations.onset[include_mask],
          duration=raw.annotations.duration[include_mask],
          description=raw.annotations.description[include_mask],
      )
      raw.set_annotations(included_annotations)

      # setting default eeg reference
      raw, _ = mne.io.set_eeg_reference(
          raw, ref_channels=[], copy=False, verbose=False
      )

      # extract events from the annotations and exclude any events not considered for epoching
      events, events_id = mne.events_from_annotations(
          raw=raw,
          event_id=None,
          verbose=False,
      )
      events = events[events[:, 2] <= len(include_annots)]
      event_id = {include_annots[i]: i + 1 for i in range(len(include_annots))}

      # specify the time window for the epochs
      tmin, tmax = -0.05, 0.95

      epochs = mne.Epochs(
          raw=raw,
          events=events,
          event_id=event_id,
          tmin=tmin,
          tmax=tmax,
          baseline=(None, 0),
          preload=True,
          verbose=False,
      )

  return epochs

In [59]:
def faster_artifact_correction(epochs):
    # Clean the data using FASTER

    # Step 1: mark bad channels
    epochs.info["bads"] = find_bad_channels(
        epochs=epochs,
        eeg_ref_corr=False,
    )
    if len(epochs.info["bads"]) > 0:
        epochs.interpolate_bads()

    # Step 2: mark bad epochs
    bad_epochs = find_bad_epochs(
        epochs=epochs,
    )
    if len(bad_epochs) > 0:
        epochs.drop(bad_epochs)

    # Step 3: mark bad ICA components (using the build-in MNE functionality for this)
    ica = mne.preprocessing.ICA(0.99999999).fit(epochs)
    ica.exclude = find_bad_components(
        ica,
        epochs,
        use_metrics=["kurtosis", "power_gradient", "hurst", "median_gradient"],
    )
    ica.apply(epochs)
    # Need to re-baseline data after ICA transformation
    epochs.apply_baseline(epochs.baseline)

    # Step 4: mark bad channels for each epoch and interpolate them.
    bad_channels_per_epoch = find_bad_channels_in_epochs(
        epochs=epochs,
        eeg_ref_corr=False,
    )
    for i, b in enumerate(bad_channels_per_epoch):
        if len(b) > 0:
            ep = epochs[i]
            ep.info["bads"] = b
            ep.interpolate_bads()
            epochs._data[i, :, :] = ep._data[0, :, :]

    return epochs

In [60]:
def pipeline(path, task):
  eeg = read_filter(
      filepath=path,
  )

  epochs = get_epochs(
      raw=eeg.copy(),
      task=task,
  )

  cleaned_epochs = faster_artifact_correction(
      epochs=epochs.copy(),
  )

  return cleaned_epochs

In [61]:
# list of length len(rest_files_path) with each object being an epoch.get_data().shape = [~144, 62, 500]
rest_epochs = [pipeline(rest_file, 'r') for rest_file in rest_files_path]

/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Data file name in EEG.data (RS_Beg_EO.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-28-s2-rest.fdt).


Bad channel detection on EEG channels:


/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Not setting position of 1 ecg channel found in montage:
['ECG1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.


	Bad by variance: []
	Bad by correlation: ['Fp1', 'Fp2']
	Bad by hurst: []
	Bad by kurtosis: ['Fp1', 'Fp2']
	Bad by line_noise: []
Interpolating bad channels
    Automatic origin fit: head of radius 93.8 mm
Computing interpolation matrix from 60 sensor positions
Interpolating 2 sensors
Bad epoch detection on EEG channels:
	Bad by amplitude: []
	Bad by deviation: [123]
	Bad by variance: [124 125 126]
Dropped 4 epochs: 123, 124, 125, 126
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by explained variance: 60 components
Fitting ICA took 57.3s.


/home/rahul/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.


...

Bad channel detection on EEG channels:
	Bad by variance: []
	Bad by correlation: ['Fp1', 'Fp2']
	Bad by hurst: []
	Bad by kurtosis: []
	Bad by line_noise: ['T7', 'O1']
Interpolating bad channels
    Automatic origin fit: head of radius 93.8 mm
Computing interpolation matrix from 58 sensor positions
Interpolating 4 sensors
Bad epoch detection on EEG channels:
	Bad by amplitude: [95 96 97]
	Bad by deviation: [ 97 107 129]
	Bad by variance: [ 96 105 106 127 128]
Dropped 9 epochs: 95, 96, 97, 105, 106, 107, 127, 128, 129
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by explained variance: 58 components
Fitting ICA took 22.0s.
Bad by kurtosis:
	[40]
Bad by power_gradient:
	[39]
Bad by hurst:
	[0]
Bad by median_gradient:
	[0]
Applying ICA to Epochs instance
    Transforming to ICA space (58 components)
    Zeroing out 3 ICA components
    Projecting back using 62 PCA components
No baseline correction applied
Bad channel-in-epoch detection on EEG c

/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Data file name in EEG.data (RS_Beg_EO.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-28-s3-rest.fdt).
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Not setting position of 1 ecg channel found in montage:
['ECG1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.


Bad channel detection on EEG channels:
	Bad by variance: []
	Bad by correlation: ['Fp1']
	Bad by hurst: []
	Bad by kurtosis: ['Fp2']
	Bad by line_noise: []
Interpolating bad channels
    Automatic origin fit: head of radius 93.8 mm
Computing interpolation matrix from 60 sensor positions
Interpolating 2 sensors
Bad epoch detection on EEG channels:
	Bad by amplitude: []
	Bad by deviation: [0]
	Bad by variance: []
Dropped 1 epoch: 0
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by explained variance: 60 components
Fitting ICA took 61.4s.
Bad by kurtosis:
	[35 38]


/home/rahul/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.


Bad by power_gradient:
	[0]
Bad by hurst:
	[18]
Bad by median_gradient:
	[1]
Applying ICA to Epochs instance
    Transforming to ICA space (60 components)
    Zeroing out 5 ICA components
    Projecting back using 62 PCA components
No baseline correction applied
Bad channel-in-epoch detection on EEG channels:
Epoch 1, Bad by amplitude:
	['FT10', 'FT8']
Epoch 2, Bad by amplitude:
	['FT10', 'FC6', 'FT8']
Epoch 3, Bad by amplitude:
	['FC6']
Epoch 4, Bad by amplitude:
	['FC6']
Epoch 33, Bad by amplitude:
	['Fp1', 'FC6']
Epoch 34, Bad by amplitude:
	['Fp1', 'FC6']
Epoch 35, Bad by amplitude:
	['FC6', 'F8']
Epoch 72, Bad by amplitude:
	['Fp1']
Epoch 73, Bad by amplitude:
	['Fp1']
Epoch 99, Bad by amplitude:
	['AFz']
Epoch 114, Bad by amplitude:
	['Fp1']
Epoch 118, Bad by amplitude:
	['PO8']
Epoch 119, Bad by amplitude:
	['PO8']
Epoch 130, Bad by amplitude:
	['Fp2']
Bad channel-in-epoch detection on EEG channels:
Epoch 0, Bad by deviation:
	['FT9']
Epoch 1, Bad by deviation:
	['F7', 'AF7']
Ep

In [62]:
# list of length len(two_files_path) with each object being an epoch.get_data().shape = [~144, 62, 501]
twob_epochs = [pipeline(twob_file, 2) for twob_file in twob_files_path]

/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Data file name in EEG.data (twoBACK.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-28-s2-2back.fdt).
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_16493/2683847859.py:3: RuntimeWarning: Not setting position of 1 ecg channel found in montage:
['ECG1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.


Bad channel detection on EEG channels:
	Bad by variance: []
	Bad by correlation: ['Fp1', 'Fp2']
	Bad by hurst: ['Fp1']
	Bad by kurtosis: ['Fp2']
	Bad by line_noise: ['T7', 'T8']
Interpolating bad channels
    Automatic origin fit: head of radius 93.8 mm
Computing interpolation matrix from 58 sensor positions
Interpolating 4 sensors
Bad epoch detection on EEG channels:
	Bad by amplitude: [101]
	Bad by deviation: []
	Bad by variance: [ 69 134 139]
Dropped 4 epochs: 69, 101, 134, 139
Fitting ICA to data using 62 channels (please be patient, this may take a while)


/tmp/ipykernel_16493/3232485165.py:20: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.


Selecting by explained variance: 58 components
Fitting ICA took 27.2s.
Bad by kurtosis:
	[40]
Bad by power_gradient:
	[0]
Bad by hurst:
	[22]
Bad by median_gradient:
	[]
Applying ICA to Epochs instance
    Transforming to ICA space (58 components)
    Zeroing out 3 ICA components
    Projecting back using 62 PCA components
Applying baseline correction (mode: mean)
Bad channel-in-epoch detection on EEG channels:
Epoch 9, Bad by amplitude:
	['Fp2']
Epoch 10, Bad by amplitude:
	['Fp2']
Epoch 22, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 34, Bad by amplitude:
	['Fp2']
Epoch 37, Bad by amplitude:
	['Fp2']
Epoch 54, Bad by amplitude:
	['Fp2']
Epoch 62, Bad by amplitude:
	['Fp2']


/tmp/ipykernel_16493/3232485165.py:26: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.


...

Bad channel detection on EEG channels:
	Bad by variance: []
	Bad by correlation: ['Fp1']
	Bad by hurst: ['T7']
	Bad by kurtosis: ['Fp2']
	Bad by line_noise: []
Interpolating bad channels
    Automatic origin fit: head of radius 93.8 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 3 sensors
Bad epoch detection on EEG channels:
	Bad by amplitude: [16 17]
	Bad by deviation: [16 17]
	Bad by variance: [16 17]
Dropped 2 epochs: 16, 17
Fitting ICA to data using 62 channels (please be patient, this may take a while)


/tmp/ipykernel_16493/3232485165.py:20: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.


Selecting by explained variance: 59 components
Fitting ICA took 6.7s.
Bad by kurtosis:
	[50]
Bad by power_gradient:
	[4]
Bad by hurst:
	[34]
Bad by median_gradient:
	[]
Applying ICA to Epochs instance
    Transforming to ICA space (59 components)
    Zeroing out 3 ICA components
    Projecting back using 62 PCA components
Applying baseline correction (mode: mean)
Bad channel-in-epoch detection on EEG channels:
Epoch 1, Bad by amplitude:
	['Fp2']
Epoch 3, Bad by amplitude:
	['Fp2']
Epoch 10, Bad by amplitude:
	['Fp2']
Epoch 17, Bad by amplitude:
	['Fp2']
Epoch 19, Bad by amplitude:
	['Fp2']
Epoch 20, Bad by amplitude:
	['Fp2']
Epoch 22, Bad by amplitude:
	['Fp2']
Epoch 26, Bad by amplitude:
	['Fp2']
Epoch 29, Bad by amplitude:
	['Fp2']
Epoch 39, Bad by amplitude:
	['Fp2']
Epoch 41, Bad by amplitude:
	['Fp2']
Epoch 49, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 50, Bad by amplitude:
	['Fp2']
Epoch 51, Bad by amplitude:
	['FC6', 'F8']
Epoch 53, Bad by amplitude:
	['Fp2']
Epoch 55, Bad by amp

/tmp/ipykernel_16493/3232485165.py:26: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.


Epoch 56, Bad by amplitude:
	['Fp2']
Epoch 74, Bad by amplitude:
	['Fp2']
Epoch 75, Bad by amplitude:
	['Fp2']
Epoch 77, Bad by amplitude:
	['Fp2']
Epoch 78, Bad by amplitude:
	['Fp2']
Epoch 79, Bad by amplitude:
	['Fp2']
Epoch 80, Bad by amplitude:
	['Fp2']
Epoch 85, Bad by amplitude:
	['Fp2']
Epoch 87, Bad by amplitude:
	['Fp2']
Epoch 88, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 91, Bad by amplitude:
	['F7', 'AF7']
Epoch 92, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 93, Bad by amplitude:
	['Fp2']
Epoch 94, Bad by amplitude:
	['Fp2']
Epoch 96, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 98, Bad by amplitude:
	['Fp2']
Epoch 101, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 103, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 104, Bad by amplitude:
	['Fp2']
Epoch 105, Bad by amplitude:
	['Fp2']
Epoch 106, Bad by amplitude:
	['Fp2']
Epoch 107, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 108, Bad by amplitude:
	['Fp2']
Epoch 109, Bad by amplitude:
	['Fp2', 'AF8']
Epoch 110, Bad by amplitude:
	['Fp2', 'AF8']
Epoc

In [63]:
display(rest_epochs[0].get_data().shape, rest_epochs[0].info)

(140, 62, 500)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, CP5, CP1, Pz, P3, P7, ...
 chs: 62 EEG
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 1.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 62
 projs: []
 sfreq: 500.0 Hz
>

In [64]:
with open (f'./data/rest_epochs_sub_{subjectid}.pkl', 'wb') as f:
  pkl.dump(rest_epochs, f)

In [65]:
with open (f'./data/twob_epochs_sub_{subjectid}.pkl', 'wb') as f:
  pkl.dump(twob_epochs, f)

### Automated MI based Channel Selection

In [66]:
with open (f'./data/rest_epochs_sub_{subjectid}.pkl', 'rb') as f:
  rest_epochs = pkl.load(f)

In [67]:
with open (f'./data/twob_epochs_sub_{subjectid}.pkl', 'rb') as f:
  twob_epochs = pkl.load(f)

In [68]:
rest_epochs_data = [rest_epochs[i].get_data() for i in range(len(rest_epochs))]
rest_epochs_arr = np.vstack(rest_epochs_data)       # array of shape [4174, 14, 500]

In [69]:
display((rest_epochs_arr.shape))

(418, 62, 500)

In [70]:
twob_epochs_data = [twob_epochs[i].get_data() for i in range(len(twob_epochs))]
twob_epochs_arr = np.vstack(twob_epochs_data)           # oneb_epochs_arr.shape = [4218, 14, 501]
twob_epochs_arr = twob_epochs_arr[:,:,:500]             # trimming the last dimension to change shape to [4218, 14, 500]

In [71]:
display(twob_epochs_arr.shape)

(419, 62, 500)

In [72]:
rest_lables  = [0] * len(rest_epochs_arr)
# zerob_lables = [1] * len(zerob_epochs_arr)
# oneb_labels  = [2] * len(oneb_epochs_arr)
twob_labels  = [1] * len(twob_epochs_arr)

In [73]:
# Combining to add tuples wise 4174+4218+4218+4217 = 16827 final shape [8392, 62, 500]
data_array = np.vstack((
                        rest_epochs_arr, 
                        # zerob_epochs_arr, 
                        # oneb_epochs_arr, 
                        twob_epochs_arr
                      ))    

In [74]:
label_array = np.hstack((
                          rest_lables, 
                          # zerob_lables, 
                          # oneb_labels, 
                          twob_labels                        
                        ))

In [75]:
with open(f'./data/data_array_sub_{subjectid}.pkl', 'wb') as f:
   pkl.dump(data_array, f)

In [76]:
with open(f'./data/label_array_sub_{subjectid}.pkl', 'wb') as f:
   pkl.dump(label_array, f)

In [77]:
with open(f'./data/data_array_sub_{subjectid}.pkl', 'rb') as f:
  data_array = pkl.load(f)

In [78]:
with open(f'./data/label_array_sub_{subjectid}.pkl', 'rb') as f:
  label_array = pkl.load(f)

In [79]:
with open('ranked_channels_all.pkl', 'rb') as f:
  ranked_channel_set = pkl.load(f)

In [80]:
#Selecting the top 14 channels with highest mutual information 
ranked_channels = ranked_channel_set[:14]
display(len(ranked_channels), ranked_channels)

14

['AF3',
 'Fp2',
 'AF8',
 'AF7',
 'T8',
 'Fp1',
 'AF4',
 'AFz',
 'FC6',
 'F1',
 'F5',
 'FC1',
 'F6',
 'C6']

In [81]:
with open(f'./data/rest_epochs_sub_{subjectid}.pkl', 'rb') as f:
  rest_epochs = pkl.load(f)

In [82]:
with open(f'./data/twob_epochs_sub_{subjectid}.pkl', 'rb') as f:
  twob_epochs = pkl.load(f)

In [83]:
rest_epochs_2 = [rest_epochs[i].pick_channels(ranked_channels) for i in range(len(rest_epochs))]
# zerob_epochs_2 = [zerob_epochs[i].pick_channels(ranked_channels) for i in range(len(zerob_epochs))]
# oneb_epochs_2 = [oneb_epochs[i].pick_channels(ranked_channels) for i in range(len(oneb_epochs))]
twob_epochs_2 = [twob_epochs[i].pick_channels(ranked_channels) for i in range(len(twob_epochs))]

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_16493/3130559692.py:1: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
/tmp/ipykernel_16493/3130559692.py:1: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
/tmp/ipykernel_16493/3130559692.py:1: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
/tmp/ipykernel_16493/3130559692.py:4: FutureWarning: The default fo

In [84]:
rest_epochs_data_2 = [rest_epochs_2[i].get_data() for i in range(len(rest_epochs_2))]
rest_epochs_arr_2 = np.vstack(rest_epochs_data_2)      # array of shape [4174, 14, 500]

In [85]:
twob_epochs_data_2 = [twob_epochs_2[i].get_data() for i in range(len(twob_epochs_2))]
twob_epochs_arr_2 = np.vstack(twob_epochs_data_2)       # oneb_epochs_arr.shape = [4218, 14, 501]
twob_epochs_arr_2 = twob_epochs_arr_2[:,:,:500]         # trimming the last dimension to change shape to [4218, 14, 500]

In [86]:
rest_lables_2  = [0] * len(rest_epochs_arr_2)
# zerob_labels_2 = [1] * len(zerob_epochs_arr_2)
# oneb_labels_2  = [2] * len(oneb_epochs_arr_2)
twob_labels_2  = [1] * len(twob_epochs_arr_2)

In [87]:
data_array_2 = np.vstack((
                        rest_epochs_arr_2, 
                        # zerob_epochs_arr_2, 
                        # oneb_epochs_arr_2, 
                        twob_epochs_arr_2
                      ))      # Combining to add tuples wise 4218+4174 = 8392 final shape [8392, 62, 500]
label_array_2 = np.hstack((
                          rest_lables_2, 
                          # zerob_lables_2, 
                          # oneb_labels_2, 
                          twob_labels_2                        
                        ))

In [88]:
np.save(f'./data//filtered-sub-{subjectid}.npy', data_array_2)
np.save(f'./data//label-sub-{subjectid}.npy', label_array_2)